# Regresión lineal 

En este cuaderno programaremos nuestra primera regresión lineal sobre una base de datos de Facebook. A partir de las distintas variables vamos a intentar predecir el número de likes de una publicación. Emplearemos los siguientes módulos:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

En este notebook omitiremos las advertencias para evitar sobrecargar la pantalla con información innecesaria. Si deseas ver las advertencias simplemente no ejecutes la siguiente celda. __Las advertencias no son lo mismo que los errores.__ En ocasiones puede aparecernos una advertencia pero si estamos seguros de lo que estamos haciendo no es un problema. Un error por el contrario corta el flujo de ejecución y debe solventarse.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Carga de datos

Comenzamos cargando nuestros datos a partir del archivo csv:

In [ ]:
data_facebook = pd.read_csv('./data/dataset_Facebook.csv', delimiter = ';')

Observamos que los datos se han cargado de manera correcta y comprobamos la cantidad de datos de la que disponemos:

In [ ]:
data_facebook.head()

In [ ]:
data_facebook.shape

La base de datos consta de 500 observaciones y 19 variables. Observemos las variables con algo más de atención:

In [ ]:
data_facebook.columns # columns nos devuelve el nombre de las columnas de la base de datos

### Diccionario de datos

A continuación se presenta el diccionario de datos con el significado de cada variable:

* **Page total likes**. El número de likes que acumula la página.
* **Type**. El tipo de publicación si es una foto, un vídeo, un status.
* **Category**. La categoría en la que se enmarca que viene codificada por un entero.
* **Post Month**. El mes en el que se realiza la publicación.
* **Post Weekeday**. El día de la semana en el que se realiza la publicación.
* **Post Hour**. La hora en la que se realiza la publicación.
* **Paid**. Si el post está pagado o no.
* **Lifetime Post Total Reach**. A cuánta gente alcanza el post en total.
* **Lifetime Post Total Impressions**. Cuántas impresiones recibe el post en total.
* **Lifetime Post Consumers**. Número de personas a las que aparece el post.
* **Lifetime Post Consumptions**. Número de veces que aparece el post.
* **Lifetime Post Impressions by people who have liked your Page**. Impresiones por personas que te siguen.
* **Lifetime Post reach by people who like your Page**. Alcance a personas que te siguen.
* **Lifetime People who have liked your Page and engaged with your post**. Número de personas que comienzan a seguirte debido a ese post.
* **Comment**. Número de comentarios en el post.
* **Like**. Número de me gustas en el post.
* **Share**. Número de veces que se comparta el post.
* **Total Interactions**. Número total de interacciones del post.

Si observamos las variables del 7 al 15 son variables __a posteriori__, es decir, cuando subimos un nuevo post no sabemos cuánto consumo va a tener o cuál va a ser el número total de impresiones por lo que descartamos dichas variables. ¿Qué sentido tendría para predecir los likes de una foto usar datos que solo se sabrán cuando hayamos subido la foto?

Descartamos dichas variables:

In [ ]:
data_facebook.drop(data_facebook.columns[7:15], axis=1,inplace=True)  #drop nos permite eliminar columnas con axis=1 y filas con axis=0

In [ ]:
data_facebook.head()

Igualmente tampoco conocemos el número de veces que se comparte, los comentarios y el número total de las interacciones por lo que las eliminamos:

In [ ]:
final_variables = data_facebook[['Page total likes', 'Type', 'Category', 'Post Month', 'Post Weekday', 'Post Hour', 'Paid', 'like']] #seleccionamos las variables que nos interesan

In [ ]:
final_variables.head()

Este es nuestro conjunto final de variables. Procedemos al análisis exploratorio.

## Análisis Exploratorio

### Variables cuantitativas

Las variables continuas son `Page total likes` y `like`:

In [ ]:
final_variables['Page total likes']

Reutilizamos la función estudiada en el notebook de análisis exploratorio:

In [ ]:
def plot_quantitative_variables(dataframe, list_quantitative_columns):
    for variable in list_quantitative_columns:
        plt.pyplot.figure()
        dataframe[variable].plot(kind = 'hist', title=variable)

In [ ]:
quantitative_variables = ['Page total likes', 'like']

In [ ]:
plot_quantitative_variables(final_variables, quantitative_variables)

Observamos que en cuanto a la distribución de likes casi todos se concentran entre los 1000 y los 2000 aunque existen unos cuantos también en 5000.

A primera vista podemos pensar que las publicaciones son de influencers y personas famosas viendo el número total de likes acumulados. Sin embargo al observar la distrbución de likes los números indican que son publicaciones que no tienen tantos likes. 

Numéricamente podemos observar que los datos están bastante bien distribuidos pues la media y la mediana se encuentran muy próximos:

In [ ]:
final_variables['Page total likes'].describe()

In [ ]:
final_variables['like'].describe()

Esta observación nos permite ver que el número medio de likes es 177 lo cual descarta nuestra idea inicial de que eran posts de famosos e influencers.

A continuación vemos las variables categóricas:

### Variables categóricas

El resto de variables son categóricas:

In [ ]:
def plot_categorical_variables(dataframe, list_categorical_columns):
    for variable in list_categorical_columns:
        plt.pyplot.figure()
        dataframe[variable].value_counts().sort_index().plot(kind='bar', title=variable)

In [ ]:
categorical_variables = ['Type', 'Category', 'Post Month', 'Post Weekday', 'Post Hour', 'Paid']

In [ ]:
plot_categorical_variables(final_variables, categorical_variables)

Analizando variable por variable observamos que:

* Con diferencia el tipo de contenido más habitual son fotografías siendo los vídeos muy poco frecuentes.
* Que las tres posibles categorías se encuentran más o menos equilibradas.
* Que no se observa una tendencia clara en las publicaciones por mes.
* Que los posts se encuentran más o menos distribuidos a lo largo de la semana con una pequeña superioridad del fin de semana.
* Que las principales horas para estos posts se han encontrado en la mañana y en la madrugada.
* Los posts no pagados son mucho más frecuentes que los no pagados. Estas clases no se encuentran equilibradas.

In [ ]:
final_variables.head()

### Gestión de valores no definidos

Observamos si existen valores no definidos:

In [ ]:
final_variables.info()

Uno o dos registros tienen una variable no definida. Podemos eliminar estas dos filas sin perder mucha información:

In [ ]:
final_variables.dropna(inplace=True)

In [ ]:
final_variables.shape

## Ingeniería de variables

En ocasiones resulta interesante a fin de dar más claridad a los datos cambiar el nombre de algunas categorías o categorizarlas de forma distintas. Además para trabajar con un modelo de regresión lineal es más útil generar variables dummy tal y cómo vimos en las secciones teóricas. En esta sección nos ocuparemos de dichas transformaciones:

### Tipo de publicación

Dividimos esta variable en tres dummies (usando cuatro habría problemas de multicolinearidad) para sus cuatro posibles valores:

In [ ]:
final_variables['Video'] = pd.get_dummies(final_variables['Type'])['Video']
final_variables['Status'] = pd.get_dummies(final_variables['Type'])['Status']
final_variables['Photo'] = pd.get_dummies(final_variables['Type'])['Photo']

In [ ]:
final_variables.head()

Los valores de enlace quedan codificados como un cero en las tres nuevas variables.

### Like

Como los likes son enteros los almacenamos en una variable entera en lugar de un float:

In [ ]:
final_variables['like'] = final_variables['like'].astype('int32')

### Categoría

Aplicamos un razonamiento análogo al de tipo de publicación:

In [ ]:
final_variables['Cat_1'] = pd.get_dummies(final_variables['Category'])[1]
final_variables['Cat_2'] = pd.get_dummies(final_variables['Category'])[2]

In [ ]:
final_variables.head()

### Mes del post

Generamos las variables dummy:

In [ ]:
month_variables = pd.get_dummies(final_variables['Post Month'],prefix='Mo')

In [ ]:
month_variables.head()

Eliminamos el representante de diciembre de nuevo para evitar colinearidad:

In [ ]:
month_variables.drop(month_variables.columns[-1], axis=1, inplace=True)

Adjuntamos las nuevas variables al dataset inicial:

In [ ]:
final_variables = pd.concat([final_variables,month_variables],axis=1) # concat concatena los dataframes

In [ ]:
final_variables.head()

### Día de semana del post

En este caso vamos a cambiar los días por sus nombres para poder observarlos mejor. 

__Nota.__ Esto sería posible hacerlo también con la variable de los meses, puedes probar a hacerlo tu mismo con un esquema análogo a este.

Definimos una función que sustituye el número por el nombre del día:

In [ ]:
def replace_day(x):
    if x == 1:
        return 'Domingo'
    elif x== 2:
        return 'Lunes'
    elif x == 3:
        return 'Martes'
    elif x == 4:
        return 'Miércoles'
    elif x == 5:
        return 'Jueves'
    elif x ==6:
        return 'Viernes'
    elif x == 7:
        return "Sábado"

Aplicamo la función a la columna y observamos el resultado:

In [ ]:
final_variables['Post Weekday'] = final_variables['Post Weekday'].apply(lambda x: replace_day(x))  # el método apply aplica la función a cada elemento de la columna Post Weekday

In [ ]:
final_variables.head()

Generamos las dummies: 

In [ ]:
weekday_variables = pd.get_dummies(final_variables['Post Weekday'])

In [ ]:
weekday_variables.head()

Eliminamos la del viernes para evitar colinearidad:

In [ ]:
weekday_variables.drop(weekday_variables.columns[-1], axis=1, inplace=True)

Las añadimos a las ya existentes:

In [ ]:
final_variables = pd.concat([final_variables,weekday_variables],axis=1)

In [ ]:
final_variables.head()

### Hora del post

De nuevo el mismo proceso:

In [ ]:
hour_variables = pd.get_dummies(final_variables['Post Hour'], prefix='Hour')

Las añadimos a nuestras variables. Como siempre eliminamos la última para evitar multicolinearidad y las añadimos:

In [ ]:
hour_variables.drop(hour_variables.columns[-1], axis=1, inplace=True)

In [ ]:
final_variables = pd.concat([final_variables,hour_variables],axis=1)

In [ ]:
final_variables.head()

### Pagado

Convertimos esta variable a string con dos posibles valores: `Sí` cuando esté pagada y `No` cuando no:

In [ ]:
def replace_pay(x):
    if int(x)==1:
        return 'Sí'
    else:
        return 'No'

Aplicamos a la columna la función de sustitución:

In [ ]:
final_variables['Paid'] = final_variables['Paid'].apply(lambda x: replace_pay(x))

Generamos una sola dummy:

In [ ]:
final_variables['Is_Paid'] = pd.get_dummies(final_variables['Paid'])['Sí']

In [ ]:
final_variables.head()

Finalmente eliminamos las variables categóricas iniciales pues ya tenemos la información de las mismas almacenadas en las variables dummies:

In [ ]:
final_variables.drop(['Type', 'Category', 'Post Month', 'Post Weekday', 'Post Hour', 'Paid'],axis=1, inplace=True)

In [ ]:
final_variables.head()

Estas son nuestras variables finales tras la ingeniería de variables:

In [ ]:
final_variables.columns

Hasta aquí el proceso inicial de limpieza de datos e ingeniería de variables. Procedemos a continuación a guardar una copia de los datos en un archivo csv. Esto es muy práctico sobre todo cuando se trabaja con grandes cantidades de datos y el proceso de limpieza lleva minutos o incluso horas. También si planeamos reutilizar el conjunto de datos en un futuro:

In [ ]:
final_variables.to_csv('./data/cleaned_facebook.csv', index=False)

## Construcción del modelo

### Instalando scikit-learn

El módulo más popular para la construcción de modelos de Machine Learning en Python es sin duda `scikit.learn` que implemente distintos modelos tanto de clasificación como de regresión así como funciones ya implementadas y optimizadas para realizar tareas como la separación de los datos en datos de entrenamiento y de evaluación o el cómputo de distintas métricas. Es necesario en primer lugar instalar el módulo:

In [ ]:
#!pip install scikit-learn

En este notebook emplearemos las siguientes herramientas contenidas en el módulo:

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

### Supresión de outliers

Por últimos vamos a eliminar los datos atípicos en la variable a predecir. Para ello simplemente suprimimos aquellas filas en las que los valores de la variable `like` están por encima del percentil 95. Para ello calculamos dicho percentil:

In [ ]:
outlierCut = np.percentile(final_variables['like'],95) #calculamos el percentil 95
outlierCut

Prescindimos de dichas filas:

In [ ]:
final_variables = final_variables[final_variables['like']<outlierCut] #nos quedamos con los valores que tienen menos del percentil 95

In [ ]:
final_variables.shape

### Separación de los datos

Como hemos visto en las secciones teóricas para evaluar un modelo de manera correcta es necesario mantener una parte de los datos oculta al modelo durante el entrenamiento. A continuación se procede a la separación en variables predictoras y variable objetivo y la separación en dos conjuntos uno de entrenamiento y otro de validación:

Almacenamos las variables predictoras en un dataframe X y la variable objetivo en un dataframe y:

In [ ]:
X = final_variables.drop(['like'], axis=1)

In [ ]:
X.head()

In [ ]:
y = final_variables[['like']]

In [ ]:
y

El método `train_test_split` nos permite dividir nuestro conjunto de variables predictoras y variable objetivo en entrenamiento y validación generando X_train con los registros de las variables predictores destinados al entrenamieno, y_train con los registros de la variable objetivo para entrenamiento, X_test con los de predictoras destinados a validación y y_test con los de objetivo destinados a validación:

In [ ]:
x_train,x_test,y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

### Escalado de variables continuas 

Escalamos la variable numérica para lograr un modelo con coeficientes mejor ajustados:

In [ ]:
from numpy import asarray

__Nota.__ Por tratarse de una única variable (normalmente  el escalado se hace de varias) es necesario cambiar algunos atributos para el buen funcionamiento, por eso compartimos la variable en un array y le cambios la forma con el método `reshape()`.

In [ ]:
scaler_train = StandardScaler() #instanciamos el escalador
scaler_train.fit(asarray(x_train['Page total likes']).reshape(-1, 1))
x_train['Page total likes'] = scaler_train.transform(asarray(x_train['Page total likes']).reshape(-1,1)) #escalamos

In [ ]:
scaler_test = StandardScaler()
scaler_test.fit(asarray(x_test['Page total likes']).reshape(-1, 1))
x_test['Page total likes'] = scaler_test.transform(asarray(x_test['Page total likes']).reshape(-1,1))

Observamos que se ha escalado correctamente. Como se centra en torno a 0 aparecen valores negativos, es totalmente normal:

In [ ]:
x_train.head()

In [ ]:
x_test.head()

Tras todo el proceso de limpieza de datos ha llegado el momento de construir nuestro primer modelo.

### Regresión lineal simple

Comenzamos construytendo el modelo más simple en el que intentamos predecir el número de likes a partir de una sola variable, por ejemplo, `Page total likes`:

In [ ]:
x_train_total_likes = x_train['Page total likes']
x_test_total_likes = x_test['Page total likes']

Empleamos la clase `linear_model` de Scikit-Learn para generar nuestro modelo que almacenamos en `simple_reg`:

In [ ]:
simple_reg = linear_model.LinearRegression(normalize=True)

Entrenamos el modelo dándole los datos sobre el total de likes en la variable `x_train_total_likes` y los datos de likes (la etiqueta) en la variable `y_train`:

In [ ]:
simple_reg.fit(asarray(x_train_total_likes).reshape(-1,1) ,y_train) #el método fit calcula los parámetros a partir de los datos provistos

Nuestro modelo ya está entrenado, es decir, ya hemos calculado los coeficientes del modelo. En este caso por ser una regresión simple tenemos el término constante:

In [ ]:
simple_reg.intercept_[0]

y la pendiente de la recta:

In [ ]:
simple_reg.coef_[0][0]

Por lo que la ecuación final de nuestra regresión sería:
<i><center>y = 133198.5839290403 * x + 1015423.7612912263</center></i>


A partir de este modelo ya entrenado podemos realizar una predicción sobre el conjunto de validación:

In [ ]:
y_pred_simple = simple_reg.predict(asarray(x_test_total_likes).reshape(-1,1)) #el método predict genera predicciones a partir de un modelo

In [ ]:
y_pred_simple

Podemos como primer método de validación y por tratarse de una regresión lineal simple graficar las dos variables (variable predictora y variable objetivo) al igual que la recta de la regresión lineal para ver cómo funciona en validación:

In [ ]:
plt.pyplot.scatter(x_test_total_likes, y_test,  color='orange', alpha = 0.8)
plt.pyplot.plot(x_test_total_likes, y_pred_simple, color='black', linewidth=1)
plt.pyplot.xlabel('Page Total Likes')
plt.pyplot.ylabel('Likes')


plt.pyplot.show()

__Nota.__ No te asustes por los valores de `Page Total Likes` recuerda que los habíamos escalado.

Por último es posible calcular el R2 asociado:

In [ ]:
y_pred_simple_train = simple_reg.predict(asarray(x_train_total_likes).reshape(-1,1))

In [ ]:
trainScore = r2_score(y_pred=y_pred_simple_train,y_true=y_train) #r2_Score nos permite calcular el R2 asociado
trainScore

In [ ]:
testScore_simple = r2_score(y_pred=y_pred_simple,y_true=y_test)
testScore_simple

### Regresión lineal múltiple. Modelo básico

Empleamos la clase `linear_model` de Scikit-Learn para generar nuestro modelo que almacenamos en basic_reg. En este caso ya trabajamos con todas las variables predictoras disponibles en lugar de solo con el Page Total Likes como ahora:

In [ ]:
basic_reg = linear_model.LinearRegression(normalize=True)

Una vez instanciado el modelo, procedemos a entrenarlo mediante el método `fit()` que recibe como parámetros los datos de entrenamiento y la etiqueta de dichos datos:

In [ ]:
basic_reg.fit(x_train,y_train)

Con estas dos simples líneas ya hemos construido nuestro modelo de regresión lineal múltiple, mediante el atributo `coef_` podemos observar cuáles son sus coeficientes estimados:

In [ ]:
basic_reg.coef_[0]

Para mejor comprensión podemos hacer una tabla para ver con que variable va asociado cada coeficiente:

In [ ]:
variables = X.columns.tolist()
variable_coef= {'Variable' : variables, 'Coeficiente': basic_reg.coef_[0]}
variable_coef_tabla = pd.DataFrame(variable_coef)
variable_coef_tabla

Hasta aquí hemos construido nuestro segundo modelo. A continuación vamos a evaluar su calidad.

## Evaluación del modelo

Para evaluar este modelo recurrimos al conjunto de datos que dejamos apartado para validación:

In [ ]:
y_pred_basic_train = basic_reg.predict(x_train)

In [ ]:
y_pred_basic = basic_reg.predict(x_test)

In [ ]:
trainScore_basic = r2_score(y_pred=y_pred_basic_train,y_true=y_train)
trainScore_basic

In [ ]:
testScore_basic = r2_score(y_pred=y_pred_basic,y_true=y_test)
testScore_basic

## Regularización

Como vimos en las secciones teóricas la regularización nos permite hacer nuestro modelo más preciso y robusto a pequeñas variaciones en los datos. En este caso examinaremos tres posibilidades: _regresión ridge, regresión lasso_ y _elastic net_:

### Regresión Ridge

El propio `scikit-learn` nos aporta el método para la regresión ridge implementado. Lo implementamos con un abanico de posibles alphas:

In [ ]:
ridge_reg = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))

Recurrimos al mismo método para entrenar el modelo:

In [ ]:
ridge_reg.fit(x_train,y_train)

Mediante validación cruzada el modelo ha obtenido el alpha óptimo:

In [ ]:
ridge_reg.alpha_

Una vez entrenado el modelo podemos emplear el conjunto de validación para evaluarlo:

In [ ]:
y_pred_ridge_train = ridge_reg.predict(x_train)

In [ ]:
trainScore_ridge = r2_score(y_pred=y_pred_ridge_train,y_true=y_train)
trainScore_ridge

In [ ]:
y_pred_ridge_test = ridge_reg.predict(x_test)

In [ ]:
testScore_ridge = r2_score(y_pred=y_pred_ridge_test,y_true=y_test)
testScore_ridge

### Regresión Lasso

En esta ocasión calculamos de manera análoga una regresión usando regularización Lasso:

In [ ]:
lasso_reg = linear_model.LassoCV(alphas=np.linspace(0,1,20))

In [ ]:
lasso_reg.fit(x_train,y_train)

El alpha ideal ha sido:

In [ ]:
lasso_reg.alpha_

Como siempre evaluamos la calidad en validación:

In [ ]:
y_pred_lasso_train = lasso_reg.predict(x_train)

In [ ]:
trainScore_lasso = r2_score(y_pred=y_pred_lasso_train,y_true=y_train)
trainScore_lasso

In [ ]:
y_pred_lasso_test = lasso_reg.predict(x_test)

In [ ]:
testScore_lasso = r2_score(y_pred=y_pred_lasso_test,y_true=y_test)
testScore_lasso

### Elastic net

Por último probaremos con elastic net. Empleamos validación cruzada para elegir el parámetros alpha más adecuado:

In [ ]:
elastic_net_reg = linear_model.ElasticNetCV(cv=10)

In [ ]:
elastic_net_reg.fit(x_train,y_train)

El alpha más adecuado ha sido:

In [ ]:
elastic_net_reg.alpha_

In [ ]:
elastic_net_reg.coef_

De nuevo calculamos R2 sobre validación para evaluar la calidad del modelo:

In [ ]:
y_pred_elastic_train = elastic_net_reg.predict(x_train)

In [ ]:
trainScore_elastic = r2_score(y_pred=y_pred_elastic_train,y_true=y_train)
trainScore_elastic

In [ ]:
y_pred_elastic_test = elastic_net_reg.predict(x_test)

In [ ]:
testScore_elastic = r2_score(y_pred=y_pred_elastic_test,y_true=y_test)
testScore_elastic

## Comparar y conclusiones

Construimos una pequeña tabla en la que almacenamos los resultados obtenidos:

In [ ]:
results = {'Simple': testScore_simple, 'Múltiple': testScore_basic, 'Cresta':testScore_ridge, 'Lasso': testScore_lasso, 'Elastic net': testScore_elastic  }
pd.DataFrame.from_dict(results, orient='index', columns=['Value'])

En este caso el mejor modelo es la regresión lasso pues es aquel que tiene el mayor R2 asociado. Observamos que hay valores negativos, es porque trabajamos con R2 y no con R2 ajustado. Calculamos finalmente para evaluar su calidad el error medio cuadrático de la regresión lasso en el conjunto de validación. Ya hemos calculado previamente la predicción así que la podemos reutilizar:

In [ ]:
from sklearn import metrics
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred_lasso_test)))

De media se equivoca en unos 93 likes, si recordamos nuestros datos de likes oscilan entre 0 y 485 por lo que fallar 93 de media es bastante elevado. Es decir, tenemos un modelo base pero será necesario en secciones posteriores buscar un modelo que mejore nuestra predicción porque el modelo actual no es demasiado bueno.